In [1]:
import gradio as gr
from diffusers import DDPMScheduler, UNet2DModel, DDPMPipeline
from accelerate import Accelerator
import os
import torch

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["NVIDIA_VISIBLE_DEVICES"] = "0"

In [3]:
output_dir="model_230402"

In [4]:
model=UNet2DModel.from_pretrained("./{}/unet/".format(output_dir)).to(torch.device("cuda"))

In [6]:
# noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
noise_scheduler = DDPMScheduler(num_train_timesteps=100)

In [7]:
mixed_precision='fp16'
gradient_accumulation_steps=1

In [8]:
accelerator = Accelerator(
    mixed_precision=mixed_precision,
    gradient_accumulation_steps=gradient_accumulation_steps, 
    log_with="tensorboard",
    project_dir=os.path.join(output_dir, "logs")
)

In [9]:
pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

In [10]:
def return_images(seed1, seed2, audio):
    images = pipeline(batch_size = 1,
                      generator=torch.manual_seed(seed1+seed2)).images
    return images[0]                 

In [13]:
# def greet(name):
#     return "Hello " + name + "!"

In [14]:
demo = gr.Interface(
    fn=return_images,
    inputs=[gr.Slider(0, 100,label="Residential Density"),
            gr.Slider(0, 100,label="Employment Density"),
            gr.Audio(source="microphone")],
#     inputs=None,
#     inputs=gr.Textbox(lines=2, placeholder="Name Here..."),
    outputs="image",
)
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://29b139f46e52cea7b3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/mas/u/doorleyr/.pyenv/versions/torch_lu/lib/python3.9/site-packages/gradio/routes.py", line 393, in run_predict
    output = await app.get_blocks().process_api(
  File "/mas/u/doorleyr/.pyenv/versions/torch_lu/lib/python3.9/site-packages/gradio/blocks.py", line 1108, in process_api
    result = await self.call_function(
  File "/mas/u/doorleyr/.pyenv/versions/torch_lu/lib/python3.9/site-packages/gradio/blocks.py", line 915, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/mas/u/doorleyr/.pyenv/versions/torch_lu/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/mas/u/doorleyr/.pyenv/versions/torch_lu/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/mas/u/doorleyr/.pyenv/versions/torch_lu/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 86